In [1]:
import os

# --- Add these lines at the very top of your script ---
# This must be done BEFORE importing numpy or other scientific libraries.
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
import time
from device import Device
from device import Device
from rgf import GreensFunction
import scipy as sp
from hamiltonian import Hamiltonian
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp
from lead_self_energy import LeadSelfEnergy
from scipy.sparse import bmat, identity, random, csc_matrix
from scipy.sparse.linalg import eigsh, eigs, spsolve
import time
from NEGF_unit_generation import UnitCell


In [2]:
# Test code to isolate dangling bond effects
dev = Device(2e-9, 1e-9)
ham = Hamiltonian(dev)
lse = LeadSelfEnergy(dev, ham)

In [7]:
GF = lse.self_energy("left", 0, 0)

In [8]:
GF

array([[ 1.15616319-1.86290124e-06j, -0.99138919+1.30301442e-06j,
        -1.00876618+1.49124299e-06j, ..., -0.18526296-2.98760693e-09j,
         0.11459978-8.55097926e-08j,  0.26535014-1.14674061e-06j],
       [-0.99138919+1.30301442e-06j,  0.36770446-1.30115839e-06j,
         0.94098835-8.96218544e-07j, ...,  0.28136868-2.41579965e-07j,
        -0.10909794+2.27819051e-07j, -0.25583774+9.05539818e-07j],
       [-1.00876618+1.49124299e-06j,  0.94098835-8.96218544e-07j,
         0.37281783-1.48552326e-06j, ..., -0.0671919 +9.17925542e-08j,
        -0.06163238-6.08427480e-08j, -0.08332706+8.74938168e-07j],
       ...,
       [-0.18526296-2.98760693e-09j,  0.28136868-2.41579965e-07j,
        -0.0671919 +9.17925542e-08j, ..., -2.78308669-2.13759652e-06j,
         0.18796412+4.39240057e-07j,  1.14240316+4.17615458e-07j],
       [ 0.11459978-8.55097926e-08j, -0.10909794+2.27819051e-07j,
        -0.06163238-6.08427480e-08j, ...,  0.18796412+4.39240057e-07j,
        -1.72716874-1.27907114e-06j

In [7]:
lse.GzerozeroH_W_sparse(csc_matrix(H00), csc_matrix(H10))

array([[-1.53187936e+05+3.96803080e-06j,  1.44505405e+05-2.63622496e-07j,
        -1.51803263e+05-1.66420068e-06j, ...,
        -1.92472565e+02-1.42822842e-07j,  9.38080679e+03+1.08845416e-06j,
         1.37039271e+04+1.71350621e-06j],
       [ 1.12664112e+05-3.79352423e-06j, -1.42953804e+05+6.33765012e-07j,
         1.14379208e+05+1.18500611e-06j, ...,
         8.17926754e+02+1.40756129e-07j, -1.10116402e+04-1.16777937e-06j,
        -1.65594222e+04-1.81726682e-06j],
       [ 8.69232246e+04-7.39155803e-06j, -1.07983945e+05+3.66570021e-06j,
         9.10233591e+04-2.96180951e-06j, ...,
         3.13967317e+02+1.05098366e-07j, -9.48930037e+03-6.01452484e-07j,
        -1.35129683e+04-9.44672138e-07j],
       ...,
       [-4.86183664e-01+2.74906764e-10j, -2.04304667e+00-6.59294841e-11j,
         1.71499642e-01+1.96967775e-10j, ...,
        -1.73714083e-01+1.18760557e-12j,  1.99722325e-01-3.30050431e-11j,
         3.78523374e-01-4.81893692e-11j],
       [ 1.56380267e+00+7.41517958e-12j, -8.

In [4]:


# Create unit cell with dangling bonds
unit_cell = UnitCell(dev.unitZ, dev.unitX, orientiation=(0, 1, 2, 3), not_NEGF=True)

# Create Hamiltonian WITHOUT potential correction
H = ham.create_sparse_channel_hamlitonian(ky=0, blocks=False, unitCell=unit_cell)

# Check eigenvalues
eigvals, eigvecs = spa.linalg.eigsh(H, k=max(50, H.shape[0]-2), which='LM')
eigvals = np.sort(eigvals.real)

# Look for bandgap
print("Eigenvalues around Fermi level:")
for i, val in enumerate(eigvals):
    if -5 < val < 5:  # Look around Fermi level
        print(f"E{i}: {val:.3f} eV")

Eigenvalues around Fermi level:
E36: -4.851 eV
E37: -4.825 eV
E38: -4.605 eV
E39: -4.301 eV
E40: -4.286 eV
E41: -4.228 eV
E42: -4.192 eV
E43: -4.029 eV
E44: -4.000 eV
E45: -3.975 eV
E46: -3.936 eV
E47: -3.875 eV
E48: -3.685 eV
E49: -3.610 eV
E50: -3.525 eV
E51: -3.503 eV
E52: -3.487 eV
E53: -3.373 eV
E54: -3.352 eV
E55: -3.312 eV
E56: -3.280 eV
E57: -3.191 eV
E58: -3.174 eV
E59: -3.151 eV
E60: -3.108 eV
E61: -3.062 eV
E62: -2.984 eV
E63: -2.959 eV
E64: -2.864 eV
E65: -2.729 eV
E66: -2.645 eV
E67: -2.576 eV
E68: -2.511 eV
E69: -2.486 eV
E70: -2.444 eV
E71: -2.365 eV
E72: -2.306 eV
E73: -2.222 eV
E74: -2.179 eV
E75: -2.051 eV
E76: -1.950 eV
E77: -1.893 eV
E78: -1.826 eV
E79: -1.742 eV
E80: -1.585 eV
E81: -1.367 eV
E82: -1.248 eV
E83: -1.202 eV
E84: -1.167 eV
E85: -0.889 eV
E86: -0.835 eV
E87: -0.746 eV
E88: -0.540 eV
E89: 1.533 eV
E90: 1.646 eV
E91: 1.687 eV
E92: 1.738 eV
E93: 1.862 eV
E94: 2.014 eV
E95: 2.092 eV
E96: 2.134 eV
E97: 2.197 eV
E98: 2.282 eV
E99: 2.359 eV
E100: 2.405 eV
E101

In [3]:

eta = 1e-6  # Small broadening parameter for numerical stability



def test_GR(E):
    dev = Device(2e-9, 1e-9)
    ham = Hamiltonian(dev)
    lse = LeadSelfEnergy(device=dev, hamiltonian=ham)
    ky = 0
    H = ham.create_sparse_channel_hamlitonian(ky, blocks=False)
    sl, sr = lse.self_energy("left", E=E, ky=ky), lse.self_energy("right", E=E, ky=ky)
    H[:ham.Nz * 10, :ham.Nz * 10] += sl
    H[-ham.Nz * 10:, -ham.Nz * 10:] += sr
    # Add small imaginary part to energy
    E_complex = E #+ 1j * eta

    # Construct Green's function: G(E) = (E*I - H)^(-1)
    H_gf = csc_matrix(np.eye(H.shape[0], dtype=complex) * E_complex) - H
    I = csc_matrix(np.eye(H.shape[0], dtype=complex))

    # Solve for Green's function
    G_R = spsolve(H_gf, I)
    return G_R


E = np.linspace(-30,30, 1000)
dE = E[1]-E[0]
H = ham.create_sparse_channel_hamlitonian(0, blocks=False)
# Calculate G_R integrated over energy using multiprocessing
G_R_integrated = np.zeros((H.shape[0], H.shape[0]), dtype=complex)

print(f"Integrating Green's function over {len(E)} energy points using multiprocessing...")
start_time = time.time()

# Use multiprocessing to parallelize the calculation
with multiprocessing.Pool(processes=64) as pool:
    G_R_results = pool.map(test_GR, (E + 1e-6j))

# Sum all the Green's function matrices
for G_R in G_R_results:
    G_R_integrated += G_R * dE

end_time = time.time()
print(f"Integration completed in {end_time - start_time:.2f} seconds")
print(f"Integrated G_R matrix shape: {G_R_integrated.shape}")


Integrating Green's function over 1000 energy points using multiprocessing...


/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)
/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j,

Integration completed in 19.01 seconds
Integrated G_R matrix shape: (560, 560)


In [4]:
G_R_integrated

matrix([[-13.55628349-0.04752034j,   2.47623138-0.0119324j ,
           2.30266392+0.01417098j, ...,   1.17537372+0.05459649j,
          -3.66916956-0.02875559j,  -5.20778324+0.01019205j],
        [  3.64726468+0.00872819j,  -3.01626135-0.00322916j,
          -1.53191801-0.01593001j, ...,   0.80243909-0.0020414j ,
          -1.13680705+0.03165546j,   4.05638384-0.00356748j],
        [  0.71538658-0.00303419j,   0.39151791-0.00115143j,
          -0.16288039-0.00238955j, ...,   1.37717127+0.00263204j,
          -2.62474135+0.0034032j ,   0.85282865-0.002233j  ],
        ...,
        [ -1.60660605-0.02664308j,  -1.93596927-0.01843426j,
          -1.41316492-0.02359291j, ...,   4.22149259+0.04757138j,
           7.73993562+0.03297571j,  -2.11717472-0.00679319j],
        [ -4.06628283-0.03669822j,  -0.7255972 -0.01990344j,
          -0.57178097-0.02151959j, ...,   6.04977217+0.06206815j,
          -0.14341761+0.03135325j,  -1.41884731-0.01021772j],
        [ -0.95172971+0.00340905j,   1.347

In [6]:
100000000j * test_GR(100000000j).toarray()

array([[ 1.00000000e+000-7.84832000e-08j,
        -1.51527280e-015+1.90972120e-22j,
        -1.51527280e-015+1.73208551e-22j, ...,
         3.77753844e-102+1.49862419e-96j,
         1.03419830e-102+5.08392119e-97j,
         4.33601288e-101+1.57352066e-95j],
       [-1.51527280e-015+1.90972120e-22j,
         1.00000000e+000-1.42292500e-07j,
         2.74652241e-014+1.00000000e-07j, ...,
         5.41241506e-102+2.05856822e-96j,
         1.52322204e-102+7.07636333e-97j,
         6.19802229e-101+2.16253238e-95j],
       [-1.51527280e-015+1.73208551e-22j,
         2.74652241e-014+1.00000000e-07j,
         1.00000000e+000-1.42292500e-07j, ...,
         4.36412202e-102+1.66617501e-96j,
         1.22229930e-102+5.71141971e-97j,
         5.01917710e-101+1.76346315e-95j],
       ...,
       [ 3.77753844e-102+1.49862419e-96j,
         5.41241506e-102+2.05856822e-96j,
         4.36412202e-102+1.66617501e-96j, ...,
         1.00000000e+000-1.37895000e-07j,
        -2.30876054e-045-1.17549435e-38j,

In [7]:
DOS((2,0))

(2, 0, np.float64(-0.0))

In [ ]:
from poisson import PoissonSolver

poissonSolver = PoissonSolver(device)

In [ ]:
poissonSolver.solve_poisson_equation()



In [ ]:
device = Device()
ham = Hamiltonian(device)
rgf = GreensFunction(device, ham)

G_R, gamma1, gamma2, sigmaL, sigmaR= rgf.sparse_rgf_G_R(E=0.1, ky = 0.1)


In [ ]:
print(G_R[0].shape)

In [ ]:
time_start = time.time()
dagger = lambda A: np.conjugate(A.T)
side = "left"
Hpp_matrices = [None] * lead.P
HpP_matrices = [None] * lead.P
hPP,hPP1 = lead.get_layer_hamiltonian(lead.P, side)

HPP = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - hPP))
Hpp_matrices[-1], HpP_matrices[-1] = HPP, HPP
for i in range(lead.P - 1, 0, -1):
    
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    Hpp = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - \
        hpp - hpp1 @ Hpp_matrices[i] @ dagger(hpp1)))
    Hpp_matrices[i - 1] = Hpp
    HpP = Hpp_matrices[i - 1] @ hpp1 @ HpP_matrices[i]
    HpP_matrices[i - 1] = HpP
    
C22 = Hpp_matrices[1]
C2P = HpP_matrices[1]
C_matrices = [None] * 4
C_matrices[1] = C22
for p in range(3, lead.P + 1):
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    C_matrices[p - 1] = Hpp_matrices[p - 1] + Hpp_matrices[p -1] @ (hpp1 @ C_matrices[p -2] @ dagger(hpp1)) @ Hpp_matrices[p - 1] 

h11, h12 = lead.get_layer_hamiltonian(1, side)

XIs = h11 + h12 @ C_matrices[1] @ dagger(h12)
XI = XIs + dagger(hPP1) @ C_matrices[-1] @ hPP1
PI = h12 @ C2P @ hPP1





In [ ]:
XIs = spa.csc_matrix(XIs)
XI = spa.csc_matrix(XI)
PI = spa.csc_matrix(PI)
I = np.eye(XI.shape[0], dtype=XI)
Z = I * 0
D = lead.E * I.copy() - XI
T = -PI

A = bmat([
    [Z, I],
    [-T.conj().T, -D]
], format='csc')

B = bmat([
    [I, Z],
    [Z, T]
], format='csc')

eigenvalues, eigenvectors = eigs(A, M=B, sigma=1.0, which='LM')




def construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, n_dim, epsilon=0.1):
    abs_vals = np.abs(eigenvalues)
    

    is_propagating = np.isclose(abs_vals, 1.0)
    is_evanescent = (abs_vals < 1.0) & (abs_vals > epsilon)
    
    selected_indices = np.where(is_propagating | is_evanescent)[0]
    
    if len(selected_indices) == 0:
        return np.array([], dtype=complex), np.array([],dtype=complex)
        
    filtered_eigenvalues = eigenvalues[selected_indices]
    filtered_eigenvectors = eigenvectors[:, selected_indices]

    Lambda_plus = np.diag(filtered_eigenvalues)
    U_plus = filtered_eigenvectors[:n_dim, :]

    return U_plus, Lambda_plus

U_plus, Lambda = construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, T.shape[0], epsilon=0.1)

U_pseudo = np.linalg.pinv(U_plus)
F = U_plus @ Lambda @ U_pseudo

Y = np.linalg.solve(lead.E * I - XIs.toarray() - PI.toarray() @ F, dagger(h12.toarray()))
self_energy = h12 @ Y
time_end = time.time()
print(f"time is: {time_end - time_start}")

In [ ]:
print(self_energy)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)

In [ ]:
def surface_gf(Energy, H00, H10, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """

    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0
    first_time = True

    while err > tol:
        if first_time:
            inv_E = Helper_functions.sparse_inverse(spa.csr_matrix(Energy * I) - spa.csr_matrix(epsilon))
            first_time = False
        else:

            inv_E = np.linalg.solve(Energy * I - epsilon, I)
    
        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)


surface_gf(0, H00, H10, tol=1e-3)

In [ ]:
def surface_gf(Energy, H00 : np.ndarray, H10: np.ndarray, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """
    
    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0

    while err > tol:
        inv_E = np.linalg.solve(Energy * I - epsilon, I)

        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)

surface_gf(0, H00, H10)

In [ ]:
from helper import Helper_functions

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)
Helper_functions.sparse_inverse(-H00)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)
Helper_functions.sparse_inverse(-H00)

In [ ]:
H = device.hamiltonian.create_sparse_hamlitonian(0.1)